<a href="https://colab.research.google.com/github/pollyjuice74/REU-LDPC-Project/blob/main/LinearTranDiff.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!git clone https://github.com/pollyjuice74/REU-LDPC-Project
!git clone https://github.com/pollyjuice74/DDECC
!wget https://raw.githubusercontent.com/NVlabs/sionna/main/sionna/utils/metrics.py -O metrics.py

Cloning into 'REU-LDPC-Project'...
remote: Enumerating objects: 1084, done.
remote: Counting objects: 100% (87/87), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 1084 (delta 67), reused 45 (delta 45), pack-reused 997
Receiving objects: 100% (1084/1084), 674.71 KiB | 2.43 MiB/s, done.
Resolving deltas: 100% (667/667), done.
Cloning into 'DDECC'...
remote: Enumerating objects: 617, done.
remote: Counting objects: 100% (349/349), done.
remote: Compressing objects: 100% (149/149), done.
remote: Total 617 (delta 194), reused 340 (delta 186), pack-reused 268
Receiving objects: 100% (617/617), 1.08 GiB | 21.07 MiB/s, done.
Resolving deltas: 100% (279/279), done.
--2024-07-16 23:56:29--  https://raw.githubusercontent.com/NVlabs/sionna/main/sionna/utils/metrics.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443.

In [3]:
import tensorflow as tf
import random
import numpy as np
import time

import os
if os.path.exists('REU-LDPC-Project'):
  os.rename('REU-LDPC-Project', 'REU_LDPC_Project')
os.chdir('REU_LDPC_Project/adv_nn')

from attention import *
from channel import *
from decoder import *
from dataset import *
from args import *
from model_functs import *
from models import *

from metrics import compute_ber, compute_bler
from DDECC.src.codes import EbN0_to_std

os.chdir('../../DDECC')

In [17]:
args = Args(model_type='dis') # args for decoder/discriminator
dec = Decoder(args)

EbNo_range_train = range(2, 8)
EbNo_range_test = range(5, 14)
# Standard deviation for train/test
std_train = [EbN0_to_std(ii, args.k / args.n) for ii in EbNo_range_train]
std_test = [EbN0_to_std(ii, args.k / args.n) for ii in EbNo_range_test]

scheduler = tf.keras.optimizers.schedules.CosineDecay(initial_learning_rate=args.lr, decay_steps=args.epochs) # 1000 is size of trainloader
optimizer = tf.keras.optimizers.Adam(learning_rate=scheduler)

train_loader = FEC_Dataset(args.code, sigma=std_test, zero_cw=True, length=args.traindata_len).batch(args.batch_size).shuffle(buffer_size=args.batch_size)

#                         z_cw   m 1s   1-cw     Should use zero codeword by default
dataset_types = {
              "bin_bits":(False, False, False), # Binary bits sent and recieved with some awgn
              "flip_cw": (True, False, True),   # Zero codeword flipped to a all ones vector [1,1,...,1]
              "zero_cw": (True, False, False),  # Standard zero codeword used for training
              "ones_m":  (False, True, False),  # Makes the message all ones vector and passes it to generator matrix producing codeword and pcm
              }

test_ebnos_datasets = [ [FEC_Dataset(args.code, sigma=std_test, zero_cw=zero_cw, ones_m=ones_m, flip_cw=flip_cw)
                                 for ii in range(len(std_test))]
                                 for (zero_cw, ones_m, flip_cw) in dataset_types.values() ]


for epoch in range(1, args.epochs + 1):
    print("Training Linear Transformer Diffusion Model...")
    # train_dec(dec, train_loader, optimizer, epoch,
    #           LR=scheduler(tf.Variable(0, dtype=tf.float32)).numpy(),
    #           traindata_len=args.traindata_len)

    # print comparison
    if epoch % 1 == 0:
        data = test_models(dec, test_ebnos_datasets, EbNo_range_test)
    break # from for loop

data


Training Linear Transformer Diffusion Model...

Testing on BIN_BITS
Testing  Linear Transformer Diffusion Model...
(121,)
Rev def call...
(121, 1)
Rev def call...
(121, 1)
Rev def call...
(121, 1)
Rev def call...
(121, 1)
Test EbN0=5, BER=0.0
Test EbN0=5, BLER=0.0
c:  tf.Tensor(
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0.], shape=(121,), dtype=float32)
c_hat:  tf.Tensor(
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 

UnknownError: {{function_node __wrapped__IteratorGetNext_output_types_8_device_/job:localhost/replica:0/task:0/device:CPU:0}} InvalidArgumentError: {{function_node __wrapped__MatMul_device_/job:localhost/replica:0/task:0/device:CPU:0}} In[0] and In[1] has different ndims: [77] vs. [80,121] [Op:MatMul] name: 
Traceback (most recent call last):

  File "/usr/local/lib/python3.10/dist-packages/tensorflow/python/ops/script_ops.py", line 270, in __call__
    ret = func(*args)

  File "/usr/local/lib/python3.10/dist-packages/tensorflow/python/autograph/impl/api.py", line 643, in wrapper
    return func(*args, **kwargs)

  File "/usr/local/lib/python3.10/dist-packages/tensorflow/python/data/ops/from_generator_op.py", line 198, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))

  File "<ipython-input-4-6406c82025a6>", line 21, in _generator
    c = (m @ G) % 2

  File "/usr/local/lib/python3.10/dist-packages/tensorflow/python/util/traceback_utils.py", line 153, 

In [18]:
import matplotlib.pyplot as plt
def plot_comparison(title, EbNo_range, ber, fer, diff_iters):
    plt.figure(figsize=(12, 6))

    # Plot diff_iters to decoding
    plt.subplot(1, 2, 1)
    plt.plot(EbNo_range, diff_iters, marker='o', label='diff_iters to decode', color='blue')
    plt.yscale('log')
    plt.xlabel('Eb/No (dB)')
    plt.ylabel('diff_iters to decode')
    plt.title('diff_iters to decode vs Eb/No')
    plt.grid(True, which="both", ls="--")
    plt.legend()

    # Plot BER for both models
    plt.subplot(1, 2, 2)
    plt.plot(EbNo_range, ber, marker='x', label='BER', color='green')
    plt.yscale('log')
    plt.xlabel('Eb/No (dB)')
    plt.ylabel('BER')
    plt.title('BER vs Eb/No')
    plt.grid(True, which="both", ls="--")
    plt.legend()

    # Plot FER for both models
    plt.subplot(1, 2, 3)
    plt.plot(EbNo_range, fer, marker='o', label='FER', color='red')
    plt.yscale('log')
    plt.xlabel('Eb/No (dB)')
    plt.ylabel('FER')
    plt.title('FER vs Eb/No')
    plt.grid(True, which="both", ls="--")
    plt.legend()

    # Set the overall title for the figure
    plt.suptitle(title)

    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.show()


for ix, dataset_type in enumerate(dataset_types.keys()):
    ber = data['LTDM'][ix]['ber']
    fer = data['LTDM'][ix]['bler']
    diff_iters = data['LTDM'][ix]['diff_iters']

    plot_comparison(dataset_type.upper(), EbNo_range_test, ber, fer, diff_iters)

KeyError: 'diff_iters'